# Cleaning Housing and Crime Datasets

## Import Dependencies and Read In Files

In [ ]:
# install SQLAlchemy-Utils - if necessary
# pip install SQLAlchemy-Utils

In [ ]:
# install sodapy - if necessary
# pip install sodapy

In [ ]:
#install kaggle - if necessary
# pip install kaggle

In [1]:
#import necessary dependencies
import pandas as pd
import numpy as np
from sodapy import Socrata
import time
#import kaggle
from matplotlib import pyplot as plt

# Import API token
#from config import app_token

# SQL database connnection imports
from config import password
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2

#machine learning 
import scipy.stats as stats
import statsmodels.api as sm
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.metrics import r2_score

In [2]:
## reminder to remove to use API calls when notebook is finalized
# read in csvs
raw_house_df = pd.read_csv('Resources/austinHousingData.csv')
raw_crime_df = pd.read_csv('Resources/Crime_Reports.csv')

C:\Users\Philip\anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3166: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## API Calls

In [ ]:
# #API call to get crime data - Takes about 13 minutes to run
# start = time.perf_counter()
# # create client variable with parameters
# client = Socrata('data.austintexas.gov',
#                  app_token=app_token)
# # retrieve data; look into limit ?get_all?
# results = client.get("fdj4-gpfu", limit=3000000)
# # Convert to pandas DataFrame
# raw_crime_df = pd.DataFrame.from_records(results)

# end = time.perf_counter()
# print(f'{end-start}')

In [ ]:
# #API call to get housing data
# kaggle.api.authenticate()
# kaggle.api.dataset_download_file('ericpierce/austinhousingprices', file_name='austinHousingData.csv',  path='Resources/')
# raw_house_df = pd.read_csv('Resources/austinHousingData.csv.zip')

## Housing Data

In [ ]:
# preview df 
raw_house_df.head()

In [ ]:
#check row and column count of df 
raw_house_df.shape

In [ ]:
#check dtypes
raw_house_df.dtypes

In [3]:
## are we dropping columns that are most 0 or just colums not applicable??
#Drop columns that are mostly 0 or not as applicable to analysis
house_df= raw_house_df.drop(['latestPriceSource','numOfPhotos'], axis = 1)
house_df.shape

(15171, 45)

In [ ]:
#what are the homeTypes? normalize=True provides the percentage
#With normalize set to True, returns the relative frequency by dividing all values by the sum of values.
house_df['homeType'].value_counts(normalize=True)


In [4]:
# Single Family makes up most of the data, we are going to drop the rest
house_df = house_df.loc[((house_df['homeType'] == 'Single Family'))]
# Only Austin houses
austin_house_df=house_df.loc[house_df['city'] == 'austin']
#what years were the houses sold? 
austin_house_df['latest_saleyear'].unique()

array([2018, 2020, 2019, 2021], dtype=int64)

In [ ]:
#Earliest sell date
austin_house_df['latest_saledate'].min()

In [ ]:
#Latest sale date
austin_house_df['latest_saledate'].max()

In [5]:
#Since not a lot of data available for 2021, will drop that year
austin_house_df=austin_house_df.loc[austin_house_df['latest_saleyear'] != 2021]

In [6]:
#Find zipcodes that sold 3 or less houses in 2018-2020, will not include 2021 since only 1 month made it into the dataset
austin2018_df=austin_house_df[austin_house_df['latest_saleyear']==2018]
austin2018_grouped_df=austin2018_df.groupby(['zipcode']).count().sort_values(by="city")
to_drop_2018=austin2018_grouped_df[austin2018_grouped_df['city']<=3].index.tolist()
#Find zipcodes that sold 3 or less houses in 2019
austin2019_df=austin_house_df[austin_house_df['latest_saleyear']==2019]
austin2019_grouped_df=austin2019_df.groupby(['zipcode']).count().sort_values(by="city")
to_drop_2019=austin2019_grouped_df[austin2019_grouped_df['city']<=3].index.tolist()
#Find zipcodes that sold 3 or less houses in 2020
austin2020_df=austin_house_df[austin_house_df['latest_saleyear']==2020]
austin2020_grouped_df=austin2020_df.groupby(['zipcode']).count().sort_values(by="city")
to_drop_2020=austin2020_grouped_df[austin2020_grouped_df['city']<=3].index.tolist()
#Combine zipcodes to drop
to_drop_zips=to_drop_2018+to_drop_2019+to_drop_2020
len(to_drop_zips)

10

In [7]:
#Drop zipcodes from DF
austin_house_df= austin_house_df[austin_house_df.zipcode.isin(to_drop_zips)==False]
austin_house_df.shape

(13999, 45)

In [ ]:
#check if there are null values 
austin_house_df[austin_house_df.isnull().any(axis = 1)]

In [8]:
#create a df with the relevant Austin housing information
df_housing=austin_house_df[["zipcode","propertyTaxRate","garageSpaces","parkingSpaces"
            ,"latestPrice","numPriceChanges","numOfAccessibilityFeatures",
            "numOfAppliances","numOfParkingFeatures","numOfPatioAndPorchFeatures"
            ,"numOfSecurityFeatures","numOfWaterfrontFeatures","numOfWindowFeatures",
             "numOfCommunityFeatures","lotSizeSqFt","livingAreaSqFt","numOfPrimarySchools"
              ,"numOfElementarySchools","numOfMiddleSchools","numOfHighSchools",
              "avgSchoolDistance","avgSchoolRating","avgSchoolSize",
              "MedianStudentsPerTeacher","numOfBathrooms","numOfBedrooms",
             "numOfStories"]].copy()

In [9]:
#Sort by bathroom number
sorted_df=df_housing.sort_values(by='numOfBathrooms',ascending=False)
sorted_df.head(10)

,zipcode,propertyTaxRate,garageSpaces,parkingSpaces,latestPrice,numPriceChanges,numOfAccessibilityFeatures,numOfAppliances,numOfParkingFeatures,numOfPatioAndPorchFeatures,...,numOfElementarySchools,numOfMiddleSchools,numOfHighSchools,avgSchoolDistance,avgSchoolRating,avgSchoolSize,MedianStudentsPerTeacher,numOfBathrooms,numOfBedrooms,numOfStories
2838,78731,1.98,2,2,745000.0,1,0,4,2,0,...,0,1,1,0.933333,7.000000,1483,16,27.0,4,1
2316,78732,1.98,5,5,13500000.0,2,0,0,2,0,...,0,1,1,2.200000,8.333333,1476,17,13.0,6,1
184,78732,1.98,4,4,4100000.0,3,0,8,3,0,...,0,1,1,3.433333,8.333333,1533,17,10.0,6,2
2547,78733,1.98,4,4,2795000.0,7,0,8,2,0,...,1,3,0,3.266667,8.000000,1373,14,10.0,7,3
375,78732,1.98,6,6,4950000.0,6,0,3,2,0,...,0,1,1,2.733333,8.333333,1476,17,10.0,5,2
5328,78735,1.98,7,7,2700000.0,4,0,5,3,0,...,0,1,1,4.300000,5.666667,1332,16,10.0,6,3
613,78733,1.98,3,3,2750000.0,7,0,1,3,0,...,1,3,0,3.500000,8.000000,1373,14,10.0,6,2
4753,78735,1.98,5,5,2190000.0,13,0,3,4,2,...,0,1,1,4.533333,5.666667,1332,16,9.0,6,3
4920,78735,1.98,4,4,2765000.0,10,0,3,3,0,...,0,1,1,4.866667,5.666667,1332,16,9.0,6,2
193,78732,1.98,0,0,5474997.0,4,0,5,2,1,...,0,1,1,2.766667,8.333333,1476,17,8.0,6,2


In [10]:
#Drop the bathroom outlier
df_housing.drop(2838,axis=0,inplace=True)

In [ ]:
#convert clean_df to csv 
df_housing.to_csv(r'Resources/HousingData_clean.csv')

## Crime Data

In [ ]:
#code to view all rows
pd.set_option("display.max_rows", None)
#preview data
raw_crime_df.sample(5)

In [11]:
#removing spaces in column names 
raw_crime_df.columns = raw_crime_df.columns.str.replace(' ','_')
#create a year column for occurred date 
raw_crime_df['year'] = pd.DatetimeIndex(raw_crime_df['Occurred_Date']).year
#Rename zipcode column to match both datasets
raw_crime_df.rename(columns={"Zip_Code":"zipcode"}, inplace=True)

In [12]:
#filter dataframe to be between 2018-2021 to match hosuing dataset
filtered_crime_df = raw_crime_df[raw_crime_df['year']>=2018]
filtered_crime_df = filtered_crime_df[filtered_crime_df['year']<=2020]
filtered_crime_df.shape

(311514, 28)

In [13]:
#check null values
#do we want to drop null values or do we want to drop the columns? 
filtered_crime_df.isnull().sum(axis = 0)

Incident_Number                     0
Highest_Offense_Description         0
Highest_Offense_Code                0
Family_Violence                     0
Occurred_Date_Time                  0
Occurred_Date                       0
Occurred_Time                       0
Report_Date_Time                    0
Report_Date                         0
Report_Time                         0
Location_Type                    1062
Address                             0
zipcode                          1864
Council_District                 3197
APD_Sector                       1167
APD_District                     1217
PRA                              1558
Census_Tract                     2915
Clearance_Status                32429
Clearance_Date                  32435
UCR_Category                   193836
Category_Description           193836
X-coordinate                     1461
Y-coordinate                     1461
Latitude                         5685
Longitude                        5685
Location    

In [14]:
#drop unneccessary columns for our analysis 
dropped_col_df= filtered_crime_df.drop(["Council_District","APD_Sector","APD_District","PRA","Occurred_Time","Report_Time"],axis = 1)
dropped_col_df.shape

(311514, 22)

In [15]:
#dropping nulls of important columns -- dropping latitude will remove rape occurances 
crime_clean_df = dropped_col_df.dropna(axis=0, subset=["Location_Type","zipcode","Census_Tract"
                                           ,"Latitude"])


In [16]:
#replacing nulls for UCR_Category with zero 
crime_clean_df[["UCR_Category"]] = crime_clean_df[["UCR_Category"]].fillna('0')
#replacing nulls with "None"
crime_clean_df[["Category_Description"]] = crime_clean_df[["Category_Description"]].fillna('None')
#replacing nulls with "U" for unkown 
crime_clean_df[["Clearance_Status"]] = crime_clean_df[["Clearance_Status"]].fillna('U')
#replacing nulls with a random date so its obvious on visualizaitons 
crime_clean_df[["Clearance_Date"]] = crime_clean_df[["Clearance_Date"]].fillna('01/01/1888')

C:\Users\Philip\anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [17]:
#convert occurred date time to datetime dtype
crime_clean_df['Occurred_Date_Time'] =  pd.to_datetime(crime_clean_df['Occurred_Date_Time'])
#create an hour column for the ocurred time
crime_clean_df['hour'] = pd.DatetimeIndex(crime_clean_df['Occurred_Date_Time']).hour
crime_clean_df['hour'].head()

C:\Users\Philip\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Philip\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


5     22
6     14
7     12
9     17
13    10
Name: hour, dtype: int64

In [18]:
#divide into time buckets
crime_clean_df['hour_category']=pd.cut(crime_clean_df['hour'],
    [0,12,18,21,23],
        labels=['Morning','Afternoon','Evening','Night'],
        include_lowest=True)

C:\Users\Philip\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [26]:
#TEST: filter for when category description does not equal to none - only "important crimes" according tyo FBI
crime_clean_df = crime_clean_df[crime_clean_df.Category_Description != 'None']


,Incident_Number,Highest_Offense_Description,Highest_Offense_Code,Family_Violence,Occurred_Date_Time,Occurred_Date,Report_Date_Time,Report_Date,Location_Type,Address,...,UCR_Category,Category_Description,X-coordinate,Y-coordinate,Latitude,Longitude,Location,year,hour,hour_category
7,2019990714,BURGLARY OF RESIDENCE,500,N,2019-04-09 12:06:00,04/09/2019,04/09/2019 12:06:00 PM,04/09/2019,RESIDENCE / HOME,2101 TERI RD,...,220,Burglary,3111980.0,3111980.0,30.201607,-97.751976,"(30.2016066, -97.75197579)",2019,12,Morning
20,20192450039,THEFT,600,N,2019-09-01 21:45:00,09/01/2019,09/02/2019 12:48:00 AM,09/02/2019,RESIDENCE / HOME,2206 HOLLY ST,...,23H,Theft,3121101.0,3121101.0,30.252086,-97.721731,"(30.25208635, -97.72173092)",2019,21,Evening
21,2019721051,BURGLARY OF RESIDENCE,500,N,2019-03-13 14:43:00,03/13/2019,03/13/2019 02:43:00 PM,03/13/2019,RESIDENCE / HOME,7302 SHADYWOOD DR,...,220,Burglary,3100725.0,3100725.0,30.188782,-97.787951,"(30.18878158, -97.78795069)",2019,14,Afternoon
22,20198003454,THEFT OF BICYCLE,609,N,2019-09-10 00:15:00,09/10/2019,09/10/2019 02:41:00 PM,09/10/2019,RESIDENCE / HOME,3506 SPEEDWAY,...,23H,Theft,3116746.0,3116746.0,30.300059,-97.734236,"(30.30005911, -97.73423629)",2019,0,Morning
29,20198003431,THEFT,600,N,2019-09-09 13:50:00,09/09/2019,09/12/2019 02:16:00 PM,09/12/2019,RESIDENCE / HOME,1802 WEST AVE,...,23H,Theft,3112970.0,3112970.0,30.282249,-97.746682,"(30.28224941, -97.74668193)",2019,13,Afternoon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2407297,20205046353,BURGLARY OF VEHICLE,601,N,2020-11-13 16:30:00,11/13/2020,11/13/2020 09:20:00 PM,11/13/2020,PARK / PLAYGROUND,12138 N LAMAR BLVD,...,23F,Theft,3133054.0,3133054.0,30.398495,-97.679840,"(30.3984947, -97.67984008)",2020,16,Afternoon
2407307,20201520852,THEFT BY SHOPLIFTING,607,N,2020-05-31 14:11:00,05/31/2020,05/31/2020 04:13:00 PM,05/31/2020,DEPARTMENT / DISCOUNT STORE,11200 LAKELINE MALL DR,...,23C,Theft,3092533.0,3092533.0,30.470517,-97.806518,"(30.4705168, -97.80651783)",2020,14,Afternoon
2407323,20203580493,AUTO THEFT,700,N,2020-12-23 02:30:00,12/23/2020,12/23/2020 10:09:00 AM,12/23/2020,RESIDENCE / HOME,10001 CRATER LAKE PASS,...,240,Auto Theft,3101709.0,3101709.0,30.154076,-97.785751,"(30.15407592, -97.78575052)",2020,2,Morning
2407335,2020461005,BURGLARY NON RESIDENCE,502,N,2020-02-14 18:00:00,02/14/2020,02/15/2020 03:58:00 PM,02/15/2020,RESTAURANTS,801 BARTON SPRINGS RD,...,220,Burglary,3111316.0,3111316.0,30.258910,-97.752547,"(30.25891009, -97.75254744)",2020,18,Afternoon


In [31]:
crime_clean_df['Category_Description'].unique()

array(['Burglary', 'Theft', 'Aggravated Assault', 'Robbery', 'Auto Theft',
       'Murder'], dtype=object)

In [32]:
## Count of crime per zipcode for 2018-2021
zipcode_grouped_df = crime_clean_df.groupby(['zipcode']).count()
zipcode_crimecount_df=zipcode_grouped_df['Highest_Offense_Code'].to_frame()
zipcode_crimecount_df.rename(columns={'Highest_Offense_Code':'Count_Of_Crimes_Per_Zip'}, inplace=True)
# Calculating Percentage of Crime Count per ZipCode
zipcode_crimecount_df['Percentage_Of_Total_Crime'] = (zipcode_crimecount_df['Count_Of_Crimes_Per_Zip'] / 
                  zipcode_crimecount_df['Count_Of_Crimes_Per_Zip'].sum()) * 100
  
# Show the dataframe
zipcode_crimecount_df.sort_values(by=['Count_Of_Crimes_Per_Zip'], ascending=False)

,Count_Of_Crimes_Per_Zip,Percentage_Of_Total_Crime
zipcode,,
78753.0,9413,8.190773
78758.0,8972,7.807034
78704.0,8779,7.639094
78741.0,8384,7.295383
78701.0,6330,5.508084
78723.0,6127,5.331442
78745.0,6008,5.227894
78759.0,5404,4.702320
78744.0,5089,4.428221


In [33]:
## is this supposed to be less than or equal to 0.9%?
#Sum offenses with a value of 0.0001 or higher
zipcode_crimecount_df[zipcode_crimecount_df['Percentage_Of_Total_Crime'] <= 0.9].sum()
#


Count_Of_Crimes_Per_Zip      7490.000000
Percentage_Of_Total_Crime       6.517464
dtype: float64

In [34]:
## is this supposed to be less than or equal to 4%?
#Sum offenses with a value of 0.0001 or higher
zipcode_crimecount_df[zipcode_crimecount_df['Percentage_Of_Total_Crime'] <= 4].sum()
#mediano

Count_Of_Crimes_Per_Zip      45494.00000
Percentage_Of_Total_Crime       39.58685
dtype: float64

In [35]:
#Cluster count of crimes in levels
zipcode_crimecount_df['Percentage_Of_Total_Crime']=pd.cut(zipcode_crimecount_df['Percentage_Of_Total_Crime'],
    [0,.9,4,9],
        labels=['Low_Crime','Medium_Crime','High_Crime'],
        include_lowest=True)
zipcode_crimecount_df.rename(columns={"Percentage_Of_Total_Crime":"Crime_Level"}, inplace=True)
# Show the dataframe
sorted_zip_df=zipcode_crimecount_df.sort_values(by=['Count_Of_Crimes_Per_Zip'], ascending=False)
sorted_zip_df.head(10)

,Count_Of_Crimes_Per_Zip,Crime_Level
zipcode,,
78753.0,9413,High_Crime
78758.0,8972,High_Crime
78704.0,8779,High_Crime
78741.0,8384,High_Crime
78701.0,6330,High_Crime
78723.0,6127,High_Crime
78745.0,6008,High_Crime
78759.0,5404,High_Crime
78744.0,5089,High_Crime


In [36]:
group_df = crime_clean_df.groupby(['zipcode','hour_category']).count()
dfp = group_df.pivot_table(index='zipcode', columns='hour_category', values=['Incident_Number'], aggfunc='sum')
dfp.head()
dfp.columns = dfp.columns.droplevel(0) #remove amount
dfp.columns.name = None               #remove categories
dfp.head()

,Morning,Afternoon,Evening,Night
zipcode,,,,
78610.0,1,0,1,1
78613.0,212,421,203,37
78617.0,317,155,94,57
78641.0,1,0,0,0
78652.0,12,5,6,0


In [ ]:
#Plot table
dfp.plot(kind='bar', subplots=True, rot=0, figsize=(9, 7), layout=(2, 3))
plt.tight_layout()

In [ ]:
#convert clean_df to csv 
crime_clean_df.to_csv(r'Resources/2018To2020CrimeData_clean.csv')

## Export to SQL

In [ ]:
#set parameters for sql database
params = {"host": "localhost", "user": "postgres", "password": password, "port": 5432}

In [ ]:
#create MyDB class
class MyDB(object):
    def __init__(self):
        self.params = params
    
    #create a function to create a new database in sql
    def create_new_db(self, newdb):
        user, host, port = self.params['user'], self.params['host'], self.params['port']
        pw = self.params['password']
        url = 'postgresql://{}:{}@{}:{}/{}'
        url = url.format(user, pw, host, port, newdb)
        
        self.engine = create_engine(url, client_encoding='utf8')
        if not database_exists(self.engine.url):
            create_database(self.engine.url)
    
    #create a function to export crime data to sql
    def dfpostgres(self, df, db_name, table_name, index):
        url = 'postgresql://{}:{}@{}:{}/{}'
        url = url.format(params['user'], params['password'], params['host'], params['port'], db_name) 
        conn = create_engine(url)
        df.to_sql(name=table_name, con=conn, if_exists='replace', index=index, chunksize=100)   
        
        return conn

In [ ]:
#create database
db = MyDB()
db.create_new_db('test4') #name of db

#export dataframes to SQL
db.dfpostgres(df_housing, 'test4', 'austin_housing', False)
db.dfpostgres(zipcode_crimecount_df, 'test4', 'zipcode_crimecount', True)
db.dfpostgres(crime_clean_df, 'test4', 'crime', False)

In [ ]:
#create connection 
conn = psycopg2.connect(dbname='test4', user=params['user'], password=password, host=params['host'])

#insert sql query to merge crime and housing dataframe before importing to jupyter notebook
cur = conn.cursor()
cur.execute('''
               DROP TABLE IF EXISTS housing_crime;

               CREATE TABLE housing_crime AS
               SELECT ah.zipcode,
                      ah."propertyTaxRate",
                      ah."garageSpaces",
                      ah."parkingSpaces",
                      ah."latestPrice",
                      ah."numPriceChanges",
                      ah."numOfAccessibilityFeatures",
                      ah."numOfAppliances",
                      ah."numOfParkingFeatures",
                      ah."numOfPatioAndPorchFeatures",
                      ah."numOfSecurityFeatures",
                      ah."numOfWaterfrontFeatures",
                      ah."numOfWindowFeatures",
                      ah."numOfCommunityFeatures",
                      ah."lotSizeSqFt",
                      ah."livingAreaSqFt",
                      ah."numOfBathrooms",
                      ah."numOfBedrooms",
                      ah."numOfStories",
                      ah."avgSchoolRating",
                      ah."numOfPrimarySchools",
                      ah."numOfElementarySchools",
                      ah."numOfMiddleSchools",
                      ah."numOfHighSchools", 
                      ah."avgSchoolDistance",
                      ah."avgSchoolSize",
                      ah."MedianStudentsPerTeacher",
                      cc."Count_Of_Crimes_Per_Zip",
                      cc."Crime_Level"
               FROM austin_housing as ah
               INNER JOIN zipcode_crimecount as cc
               ON ah.zipcode = cc.zipcode
               ;''')
#commit the merge to SQL
conn.commit()

#import merged table as dataframe from SQL
housing_crime_df = pd.read_sql("select * from \"housing_crime\"", conn);
crime_df = pd.read_sql("select * from \"crime\"", conn);

#close connection
cur.close()
conn.close()

In [ ]:
#show merged dataframe
housing_crime_df.head()

In [ ]:
#show crime dataframe
crime_df.head()

In [37]:
df_housing.head()

,zipcode,propertyTaxRate,garageSpaces,parkingSpaces,latestPrice,numPriceChanges,numOfAccessibilityFeatures,numOfAppliances,numOfParkingFeatures,numOfPatioAndPorchFeatures,...,numOfElementarySchools,numOfMiddleSchools,numOfHighSchools,avgSchoolDistance,avgSchoolRating,avgSchoolSize,MedianStudentsPerTeacher,numOfBathrooms,numOfBedrooms,numOfStories
17,78717,2.21,0,0,687850.0,13,0,4,2,0,...,0,1,1,3.266667,7.666667,1259,14,6.0,5,2
18,78717,2.21,2,2,409000.0,4,0,6,3,0,...,0,1,1,1.933333,8.333333,1481,16,4.0,5,2
19,78724,1.98,2,2,199000.0,3,1,2,2,1,...,0,1,1,2.433333,2.666667,1478,13,2.0,3,1
20,78717,2.21,0,0,400000.0,2,0,3,2,0,...,0,1,1,1.900000,8.333333,1481,16,2.0,3,1
22,78717,2.21,2,2,549900.0,2,0,4,3,0,...,0,1,1,3.300000,7.666667,1259,14,5.0,4,2


In [39]:
crime_clean_df.head()

,Incident_Number,Highest_Offense_Description,Highest_Offense_Code,Family_Violence,Occurred_Date_Time,Occurred_Date,Report_Date_Time,Report_Date,Location_Type,Address,...,UCR_Category,Category_Description,X-coordinate,Y-coordinate,Latitude,Longitude,Location,year,hour,hour_category
7,2019990714,BURGLARY OF RESIDENCE,500,N,2019-04-09 12:06:00,04/09/2019,04/09/2019 12:06:00 PM,04/09/2019,RESIDENCE / HOME,2101 TERI RD,...,220,Burglary,3111980.0,3111980.0,30.201607,-97.751976,"(30.2016066, -97.75197579)",2019,12,Morning
20,20192450039,THEFT,600,N,2019-09-01 21:45:00,09/01/2019,09/02/2019 12:48:00 AM,09/02/2019,RESIDENCE / HOME,2206 HOLLY ST,...,23H,Theft,3121101.0,3121101.0,30.252086,-97.721731,"(30.25208635, -97.72173092)",2019,21,Evening
21,2019721051,BURGLARY OF RESIDENCE,500,N,2019-03-13 14:43:00,03/13/2019,03/13/2019 02:43:00 PM,03/13/2019,RESIDENCE / HOME,7302 SHADYWOOD DR,...,220,Burglary,3100725.0,3100725.0,30.188782,-97.787951,"(30.18878158, -97.78795069)",2019,14,Afternoon
22,20198003454,THEFT OF BICYCLE,609,N,2019-09-10 00:15:00,09/10/2019,09/10/2019 02:41:00 PM,09/10/2019,RESIDENCE / HOME,3506 SPEEDWAY,...,23H,Theft,3116746.0,3116746.0,30.300059,-97.734236,"(30.30005911, -97.73423629)",2019,0,Morning
29,20198003431,THEFT,600,N,2019-09-09 13:50:00,09/09/2019,09/12/2019 02:16:00 PM,09/12/2019,RESIDENCE / HOME,1802 WEST AVE,...,23H,Theft,3112970.0,3112970.0,30.282249,-97.746682,"(30.28224941, -97.74668193)",2019,13,Afternoon


In [41]:
zipcode_crimecount_df.head(df_housing, )

,Count_Of_Crimes_Per_Zip,Crime_Level
zipcode,,
78610.0,3,Low_Crime
78613.0,873,Low_Crime
78617.0,623,Low_Crime
78641.0,1,Low_Crime
78652.0,23,Low_Crime


In [43]:
housing_crime_df = df_housing.merge(zipcode_crimecount_df, on='zipcode')
housing_crime_df

,zipcode,propertyTaxRate,garageSpaces,parkingSpaces,latestPrice,numPriceChanges,numOfAccessibilityFeatures,numOfAppliances,numOfParkingFeatures,numOfPatioAndPorchFeatures,...,numOfHighSchools,avgSchoolDistance,avgSchoolRating,avgSchoolSize,MedianStudentsPerTeacher,numOfBathrooms,numOfBedrooms,numOfStories,Count_Of_Crimes_Per_Zip,Crime_Level
0,78717,2.21,0,0,687850.0,13,0,4,2,0,...,1,3.266667,7.666667,1259,14,6.0,5,2,1118,Medium_Crime
1,78717,2.21,2,2,409000.0,4,0,6,3,0,...,1,1.933333,8.333333,1481,16,4.0,5,2,1118,Medium_Crime
2,78717,2.21,0,0,400000.0,2,0,3,2,0,...,1,1.900000,8.333333,1481,16,2.0,3,1,1118,Medium_Crime
3,78717,2.21,2,2,549900.0,2,0,4,3,0,...,1,3.300000,7.666667,1259,14,5.0,4,2,1118,Medium_Crime
4,78717,2.21,3,3,400000.0,2,0,3,3,2,...,1,2.033333,8.333333,1481,16,3.0,5,2,1118,Medium_Crime
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13993,78756,1.98,6,6,495000.0,5,0,3,3,0,...,1,0.766667,7.000000,1189,16,1.0,2,1,1094,Medium_Crime
13994,78756,1.98,1,1,445000.0,4,0,1,3,0,...,1,0.666667,7.000000,1189,16,1.0,3,1,1094,Medium_Crime
13995,78756,1.98,1,1,400000.0,1,0,5,2,1,...,1,0.600000,7.000000,1189,16,2.0,2,1,1094,Medium_Crime
13996,78756,1.98,0,0,369900.0,7,0,3,1,0,...,1,0.666667,7.000000,1189,16,1.0,3,1,1094,Medium_Crime


# Machine Learning Model

In [44]:
#encode zip code 
encode = pd.get_dummies(housing_crime_df['zipcode'], prefix='Zip')
df_zips = housing_crime_df.join(encode)
df_zips

,zipcode,propertyTaxRate,garageSpaces,parkingSpaces,latestPrice,numPriceChanges,numOfAccessibilityFeatures,numOfAppliances,numOfParkingFeatures,numOfPatioAndPorchFeatures,...,Zip_78749,Zip_78750,Zip_78751,Zip_78752,Zip_78753,Zip_78754,Zip_78756,Zip_78757,Zip_78758,Zip_78759
0,78717,2.21,0,0,687850.0,13,0,4,2,0,...,0,0,0,0,0,0,0,0,0,0
1,78717,2.21,2,2,409000.0,4,0,6,3,0,...,0,0,0,0,0,0,0,0,0,0
2,78717,2.21,0,0,400000.0,2,0,3,2,0,...,0,0,0,0,0,0,0,0,0,0
3,78717,2.21,2,2,549900.0,2,0,4,3,0,...,0,0,0,0,0,0,0,0,0,0
4,78717,2.21,3,3,400000.0,2,0,3,3,2,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13993,78756,1.98,6,6,495000.0,5,0,3,3,0,...,0,0,0,0,0,0,1,0,0,0
13994,78756,1.98,1,1,445000.0,4,0,1,3,0,...,0,0,0,0,0,0,1,0,0,0
13995,78756,1.98,1,1,400000.0,1,0,5,2,1,...,0,0,0,0,0,0,1,0,0,0
13996,78756,1.98,0,0,369900.0,7,0,3,1,0,...,0,0,0,0,0,0,1,0,0,0


In [45]:
# encode Crime_Level ???
encode2 = pd.get_dummies(housing_crime_df['Crime_Level'], prefix='Crime_Level')
df_zips = df_zips.join(encode2)
df_zips

,zipcode,propertyTaxRate,garageSpaces,parkingSpaces,latestPrice,numPriceChanges,numOfAccessibilityFeatures,numOfAppliances,numOfParkingFeatures,numOfPatioAndPorchFeatures,...,Zip_78752,Zip_78753,Zip_78754,Zip_78756,Zip_78757,Zip_78758,Zip_78759,Crime_Level_Low_Crime,Crime_Level_Medium_Crime,Crime_Level_High_Crime
0,78717,2.21,0,0,687850.0,13,0,4,2,0,...,0,0,0,0,0,0,0,0,1,0
1,78717,2.21,2,2,409000.0,4,0,6,3,0,...,0,0,0,0,0,0,0,0,1,0
2,78717,2.21,0,0,400000.0,2,0,3,2,0,...,0,0,0,0,0,0,0,0,1,0
3,78717,2.21,2,2,549900.0,2,0,4,3,0,...,0,0,0,0,0,0,0,0,1,0
4,78717,2.21,3,3,400000.0,2,0,3,3,2,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13993,78756,1.98,6,6,495000.0,5,0,3,3,0,...,0,0,0,1,0,0,0,0,1,0
13994,78756,1.98,1,1,445000.0,4,0,1,3,0,...,0,0,0,1,0,0,0,0,1,0
13995,78756,1.98,1,1,400000.0,1,0,5,2,1,...,0,0,0,1,0,0,0,0,1,0
13996,78756,1.98,0,0,369900.0,7,0,3,1,0,...,0,0,0,1,0,0,0,0,1,0


In [46]:
# drop columns with insignificant p-values 
df_zips = df_zips.drop(columns=['garageSpaces','parkingSpaces',
                                'numOfAccessibilityFeatures','numOfAppliances',
                     'numOfSecurityFeatures','numOfWindowFeatures','lotSizeSqFt',"Crime_Level"])

In [47]:
#check for null values 
df_zips.isnull().sum(axis = 0)

zipcode                     0
propertyTaxRate             0
latestPrice                 0
numPriceChanges             0
numOfParkingFeatures        0
                           ..
Zip_78758                   0
Zip_78759                   0
Crime_Level_Low_Crime       0
Crime_Level_Medium_Crime    0
Crime_Level_High_Crime      0
Length: 62, dtype: int64

In [48]:
# defne features and target 
y = df_zips[["latestPrice"]]
X = df_zips.drop(["latestPrice","zipcode"], axis=1)

In [49]:
# split data into train and test sets 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [50]:
# create a standard scaler instance 
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [51]:
# fit and transform standard scaler with training data
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [52]:
# create model 
model_final = linear_model.LinearRegression()

In [53]:
#fit model 
model_final.fit(X_train,y_train)

LinearRegression()

In [54]:
# make predictions
y_pred = model_final.predict(X_test)
y_pred.shape

(3500, 1)

In [55]:
#getting column names into a list 
col_list = X.columns.values.tolist()
col_list[:0] = ['const']
len(col_list)

61

In [56]:
X2 = sm.add_constant(X_train)
est = sm.OLS(y_train, X2)
est2 = est.fit()
print(est2.summary(xname=col_list))

                            OLS Regression Results                            
Dep. Variable:            latestPrice   R-squared:                       0.591
Model:                            OLS   Adj. R-squared:                  0.589
Method:                 Least Squares   F-statistic:                     274.0
Date:                Sun, 27 Mar 2022   Prob (F-statistic):               0.00
Time:                        15:36:41   Log-Likelihood:            -1.4726e+05
No. Observations:               10498   AIC:                         2.946e+05
Df Residuals:                   10442   BIC:                         2.950e+05
Df Model:                          55                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const               